# Notes on draft position statistics:

- The **CurrentRoster view** only contains for players on the 2018 roster right now. I actually don't have the ability to generate each year's rosters going back in time, ESPN doesn't make that data available anymore, which is really frustrating.

- After inheriting Cody's team, a team which missed the 2015 draft altogether, Jon was allowed to pick up any number of free agents. He "drafted" 10 of them, and they count in the 10th round of the 2015 draft.

- Counts the most recent time the player was drafted into our league, even if not by the player's current team (in the case of trades for example).

- Jon and Adrian's drafting activity does not include Cody and Patrick's.

***

## Average Draft Year for each roster.

- Todd has the oldest roster by year they were drafted, while Brian and William are using more recently drafted players than other teams.

- We all still have at least 2 players from 2012, but there are only 4 players left from the 2013 draft, and none were taken in the first round that year.

- Of course, the most recent draft will always stock 72 out of 200 players in the league, as 8 teams must draft 72 players in total.

In [19]:
select  Current_owner
,       Avg_Draft_Year = cast(round(avg(draftyear*1.),2) as decimal(9,2)) 
,       From_2012_Draft = SUM(CASE WHEN cr.draftyear = 2012 THEN 1 ELSE 0 END)
,       From_2013_Draft = SUM(CASE WHEN cr.draftyear = 2013 THEN 1 ELSE 0 END)
,       From_2014_Draft = SUM(CASE WHEN cr.draftyear = 2014 THEN 1 ELSE 0 END)
,       From_2015_Draft = SUM(CASE WHEN cr.draftyear = 2015 THEN 1 ELSE 0 END)
,       From_2016_Draft = SUM(CASE WHEN cr.draftyear = 2016 THEN 1 ELSE 0 END)
,       From_2017_Draft = SUM(CASE WHEN cr.draftyear = 2017 THEN 1 ELSE 0 END)
,       From_2018_Draft = SUM(CASE WHEN cr.draftyear = 2018 THEN 1 ELSE 0 END)
from CurrentRoster cr
where rosteryear = 2018
Group by rollup (Current_Owner)
order by isnumeric(isnull(current_owner,1)), Avg_Draft_Year;

select dr.draftyear, dr.pick, dr.Player, dr.NFLTeam, dr.Position, Drafted_Owner, Current_Owner
from draftresults dr
inner join team t_dr on dr.TeamID = t_dr.id
inner join currentroster r on r.player = dr.Player and t_dr.TeamNumber = r.Drafted_Team_Number and r.draftyear = dr.draftyear
inner join team t_c on r.Current_Team_Number = t_c.TeamNumber and t_c.BeginSeason <= r.rosteryear and t_c.EndSeason >= r.rosteryear
where dr.draftyear = 2013
and r.rosteryear = 2018 
order by draftyear, pick;

(9 rows affected)

(4 rows affected)

Total execution time: 00:00:00.097

Current_owner,Avg_Draft_Year,From_2012_Draft,From_2013_Draft,From_2014_Draft,From_2015_Draft,From_2016_Draft,From_2017_Draft,From_2018_Draft
Todd,2015.76,3,2,2,3,3,5,7
Paul,2015.96,4,1,2,1,3,5,9
Adrian,2015.96,5,0,1,2,3,5,9
Ian,2016.00,5,0,2,2,1,4,11
Christine,2016.08,4,1,1,1,3,6,9
Jon,2016.12,3,0,3,2,3,5,9
Brian,2016.20,2,0,2,5,3,4,9
William,2016.20,4,0,1,1,4,6,9
NULL,2016.04,30,4,14,17,23,40,72


draftyear,pick,Player,NFLTeam,Position,Drafted_Owner,Current_Owner
2013,2-2,DeAndre Hopkins,Hou,WR,Todd,Todd
2013,2-6,T.Y. Hilton,Ind,WR,Brian,Christine
2013,2-7,Golden Tate,Det,WR,Paul,Paul
2013,4-2,LeVeon Bell,Pit,RB,Todd,Todd


***

## Average Draft Round for each roster

- In 2018, Brian used, on average, players that were drafted two rounds higher than Adrian.

- Brian's 2018 players were picked 29th overall in the draft, 19 picks ahead of Adrian's players.

- This means Brian has cut or traded fewer high-round picks than the rest of us

In [20]:
select Current_owner
, Avg_Draft_Round = cast(round(avg(draft_round*1.),2) as decimal(9,2)) 
, Avg_Draft_pick_overall = cast(round(avg(((draft_round-1)*8)+pick_in_draft_round*1.),2) as decimal(9,2)) 
from CurrentRoster 
where rosteryear = 2018
Group by Current_Owner order by Avg_Draft_Round;

(8 rows affected)

Total execution time: 00:00:00.068

Current_owner,Avg_Draft_Round,Avg_Draft_pick_overall
Brian,4.20,29.04
Paul,4.44,32.48
Christine,4.64,34.84
William,4.92,35.52
Todd,4.96,36.52
Jon,5.56,39.68
Ian,6.16,45.16
Adrian,6.48,48.88


***
## Average number of draft picks pear year 

- Remember in the first two seasons, we only picked 8 players in the draft. 

- Only includes the first 9 rounds of picks, so this average is only affected by trades

- Doesn't include Jon's "10th round" of 10 picks after he inherited Cody's lost draft of 2015

- Doesn't count any 10th round picks used for the "Injured After Drops" rule

- Ian's clearly been trading to acquire more picks



In [21]:
with cteDraftPicksperYear (teamid, draftyear, numberofpicks) as (select teamid, draftyear, count(id) from dbo.draftresults where draft_round <= 9 group by teamid, draftyear)
select t.TeamOwner, avg_number_of_picks = cast(round(avg(numberofpicks*1.),3)  as decimal(9,2))
from cteDraftPicksperYear dr
inner join team t on t.id = dr.TeamID 
where dr.draftyear > 2012 --don't count our 25-round initial seed draft
group by t.TeamOwner
order by avg_number_of_picks;

(10 rows affected)

Total execution time: 00:00:00.054

TeamOwner,avg_number_of_picks
Cody,8.50
Patrick,8.50
Todd,8.50
William,8.83
Paul,8.83
Brian,8.83
Christine,8.83
Adrian,9.00
Jon,9.00
Ian,9.17


***

## Average pick in the round

-Mostly determined by pre-draft order of how you finish the season, but could be affected by trades or the draft party mock draft game

-Christine's two titles clearly weighing down the order in the round

In [22]:
select Current_owner, Avg_pick_in_draft_round = cast(round(avg(pick_in_draft_round*1.),2) as decimal(9,2)) 
from CurrentRoster 
where rosteryear = 2018
group by Current_Owner 
order by Avg_pick_in_draft_round;

(8 rows affected)

Total execution time: 00:00:00.128

Current_owner,Avg_pick_in_draft_round
Jon,3.20
Brian,3.44
Ian,3.88
William,4.16
Todd,4.84
Paul,4.96
Adrian,5.04
Christine,5.72


***

## Average draft round by position drafted

- William drafts RB's high much higher than every other team, not that it's done him any good. 

- Adrian drafts QB's the latest, and yet he's still stacked with QB's right now.

- Paul and William wait the longest to take TE's

- Brian drafts WR's and QB's high much higher than every other team

- Only includes the first 9 rounds, so it ignores any 10th round scenarios and the 10th+ rounds of our initial seeding draft, for consistency

In [23]:
select TeamOwner, QB = cast(QB as decimal(9,2)), RB = cast(RB as decimal(9,2)), WR = cast(WR as decimal(9,2)), TE = cast(TE as decimal(9,2)), K = cast(K as decimal(9,2)) 
from (
select t.TeamOwner, dr.position, Avg_draft_round = cast(round(avg(draft_round*1.),2) as decimal(9,2)) from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Cody','Patrick')
group by t.TeamOwner, dr.Position
) x 
PIVOT (AVG(Avg_draft_round) for Position in (RB, TE, WR, QB, K)) as pvt
ORDER BY RB asc;

(8 rows affected)

Total execution time: 00:00:00.081

TeamOwner,QB,RB,WR,TE,K
William,6.27,2.72,4.38,6.00,7.14
Paul,5.00,4.21,4.50,6.00,7.20
Todd,4.00,4.26,4.33,5.27,6.00
Jon,4.67,4.62,6.00,4.83,6.75
Christine,4.44,4.65,5.00,5.25,7.20
Ian,4.33,4.90,5.05,4.44,6.80
Adrian,6.50,5.17,4.56,3.67,6.33
Brian,3.90,5.71,3.59,5.45,7.50


***

## More on what positions people draft, and when.

- William drafts RB's much higher than everyone else, and drafts QB's later than everyone else, not that its done him any good.

- Brian drafts WR's a round higher than everyone else.

- Todd drafts a TE in the 5th-6th rounds, consistently.

- Christine is the least predictable of us when it comes to drafting, and with less sample size, so is Adrian

In [27]:
select t.TeamOwner, dr.position
, Avg_draft_round = cast(round(avg(draft_round*1.),2) as decimal(9,2))
, count_of_picks_of_this_position = count(dr.id) 
, STDEV_draft_round = cast(STDEV(draft_round*1.) as decimal(9,2)) 
from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Cody','Patrick')
and position <>'K'
group by t.TeamOwner, dr.Position
having count(dr.id)  > 3 -- not enough sample size
ORDER BY Avg_Draft_round;

(29 rows affected)

Total execution time: 00:00:00.065

TeamOwner,position,Avg_draft_round,count_of_picks_of_this_position,STDEV_draft_round
William,RB,2.72,18,1.90
Brian,WR,3.59,17,2.21
Brian,QB,3.90,10,2.08
Todd,QB,4.00,9,2.74
Paul,RB,4.21,19,2.70
Todd,RB,4.26,19,2.60
Ian,QB,4.33,9,2.87
Todd,WR,4.33,12,3.03
William,WR,4.38,16,2.00
Ian,TE,4.44,9,2.13


***

## Number of players at each position drafted + average number of that position picked per draft

- Not everyone has participated in the same number of drafts. Jon and Adrian have only been in the league 4 seasons, not all 7.

- Todd is super skeptical of WR's, drafting less than 2 per draft, far fewer than everyone else.

- Christine has drafted more WR's than anyone, but Adrian is catching up, drafting an avg of 4 per year, highest in the league.

- Thanks to Oli Beckhamming, Paul has drafted fewer QB's over 7 years than anyone else, while with Cam and Russell Wilson, Adrian has drafted few QB's too.

- In fact, Adrian has drafted mostly RB's and WR's since entering the league, higher than the rest of us.

- William has drafted the most QB's, a lot of good it's done. He did trade Dak for a kicker though.

- Brian has drafted the most kickers. This is real advanced analytics, I know.

In [25]:
WITH cteCount (TeamOwner, position, count_of_picks_of_this_position, count_of_drafts) as (
 select t.TeamOwner, dr.position
, count_of_picks_of_this_position = count(dr.id)  
, count_of_drafts = count(distinct dr.draftyear)   
from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round <= 9 
and t.TeamOwner not in ('Cody','Patrick')
group by t.TeamOwner, dr.position
) 
SELECT TeamOwner, QB, RB, WR, TE, K FROM (
select teamowner, position
,  Count_of_picks_Avg_per_draft = cast(count_of_picks_of_this_position  as varchar(4)) + ' ' +  cast(cast(count_of_picks_of_this_position*1./ MAX(count_of_drafts*1.) OVER (PARTITION BY TeamOwner) as decimal(9,2)) as varchar(6))
from cteCount 
) x
PIVOT ( MAX(Count_of_picks_Avg_per_draft) FOR Position IN (WR, RB, QB, TE, K)) AS pvt
ORDER BY WR desc;

(8 rows affected)

Total execution time: 00:00:00.082

TeamOwner,QB,RB,WR,TE,K
Christine,9 1.29,17 2.43,23 3.29,8 1.14,5 0.71
Ian,9 1.29,21 3.00,20 2.86,9 1.29,5 0.71
Paul,8 1.14,19 2.71,20 2.86,10 1.43,5 0.71
Brian,10 1.43,14 2.00,17 2.43,11 1.57,10 1.43
Adrian,2 0.50,12 3.00,16 4.00,3 0.75,3 0.75
William,11 1.57,18 2.57,16 2.29,10 1.43,7 1.00
Todd,9 1.29,19 2.71,12 1.71,11 1.57,9 1.29
Jon,3 0.75,13 3.25,10 2.50,6 1.50,4 1.00


***

## First Round Pick Behavior

- Most of us take RB's in the first round

- Jon traded a first round pick to Todd in 2017, and Brian traded a first round pick to Adrian in 2017

- The newcomers, Jon and Adrian, are the only ones to draft a TE in the first.

- William's only non-RB pick in the first round: then-Rams QB Nick Foles in 2014. Wow he is bad at this.

In [26]:
select TeamOwner, QB, RB, WR, TE, K
from (
select t.TeamOwner, dr.position, pos_count = count(player) 
from draftresults  dr
inner join team t on t.id = dr.TeamID 
where draft_round = 1 
and t.TeamOwner not in ('Cody','Patrick')
group by t.TeamOwner, dr.Position
) x 
PIVOT (AVG(pos_count) for Position in (RB, TE, WR, QB, K)) as pvt
ORDER BY RB desc;

(8 rows affected)

Total execution time: 00:00:00.062

TeamOwner,QB,RB,WR,TE,K
William,1,6,NULL,NULL,NULL
Paul,NULL,5,2,NULL,NULL
Todd,1,5,2,NULL,NULL
Christine,1,5,1,NULL,NULL
Ian,1,5,1,NULL,NULL
Adrian,NULL,2,2,1,NULL
Brian,2,2,2,NULL,NULL
Jon,NULL,1,1,1,NULL
